In [1]:
!pip install transformers datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [2]:
pip install tensorflow

In [3]:
import json
from datasets import load_dataset
from collections import Counter

ds= load_dataset('csv', data_files='AL_data.csv')

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:
def find_answer_start(example):

    start = example['context'].strip().lower().find(example['answers'].strip())
    if start == -1:
      print(example['context'].lower())
      print(example['answers'])
      print((example['context'].lower()).find(example['answers']))
      raise ValueError("No answer")

    temp = example['answers']
    example['answers'] = {}
    example['answers']['text'] = [temp]
    example['answers']['answer_start'] = [start]

    return example

In [5]:
from datasets import load_from_disk,DatasetDict

qa_dataset = ds['train'].map(find_answer_start)
qa_dataset.save_to_disk('./data')
ds = load_from_disk('./data')

ds = ds.class_encode_column("title")
train_test = ds.train_test_split(test_size=0.25,stratify_by_column="title")
# gather everyone if you want to have a single DatasetDict
ds = DatasetDict({
    'train': train_test['train'],
    'test': train_test['test'],
})
print(sorted(Counter(ds['train']['title']).items(), key=lambda pair: pair[0], reverse=True))
print(sorted(Counter(ds['test']['title']).items(), key=lambda pair: pair[0], reverse=True))

Map:   0%|          | 0/214 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/214 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/214 [00:00<?, ? examples/s]

[(5, 30), (4, 29), (3, 11), (2, 30), (1, 30), (0, 30)]
[(5, 10), (4, 10), (3, 4), (2, 10), (1, 10), (0, 10)]


In [6]:
from transformers import AutoTokenizer

# model_name = "bert-base-uncased"
#model_name = "deepset/bert-base-uncased-squad2"
model_name = "google-bert/bert-large-uncased-whole-word-masking-finetuned-squad"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.is_fast

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

True

In [7]:
context = ds["train"][0]["context"]
question = ds["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'[CLS] what should you write for your goals in each key area of your life? [SEP] in summary, developing an abundant life plan involves dividing your life into key areas, writing headlines and stories for your goals in each area, backcasting your journey from the achieved goals to your current situation, and identifying the necessary capabilities, habits, and mindsets by researching role models. [SEP]'

In [8]:
max_length = 384
stride = 64

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [9]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [10]:
tokenized_train_data = ds['train'].map(preprocess_training_examples, batched=True,remove_columns=ds["train"].column_names)
tokenized_test_data = ds['test'].map(preprocess_validation_examples, batched=True,remove_columns=ds["test"].column_names)
print(tokenized_train_data)
print(tokenized_test_data)

Map:   0%|          | 0/160 [00:00<?, ? examples/s]

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 160
})
Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 54
})


In [11]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [12]:
from transformers import create_optimizer

batch_size = 16
num_epochs = 3
total_train_steps = (len(tokenized_train_data) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
)

In [13]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(model_name)

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

All the weights of TFBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [14]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_train_data,
    shuffle=True,
    batch_size=4,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_test_data,
    shuffle=False,
    batch_size=4,
    collate_fn=data_collator,
)

In [15]:
from huggingface_hub import notebook_login

notebook_login()

In [17]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir="abundant_life_planning_qa_model",
    tokenizer=tokenizer,
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/tamarailic11/abundant_life_planning_qa_model into local empty directory.


Download file tf_model.h5:   0%|          | 16.0k/1.25G [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/1.25G [00:00<?, ?B/s]

In [18]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [19]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=10, callbacks=[callback])


Epoch 1/10


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
40/40 [==============================] - 337s 5s/step - loss: 1.4127 - val_loss: 0.0000e+00
Epoch 2/10
40/40 [==============================] - 77s 2s/step - loss: 1.0524 - val_loss: 0.0000e+00
Epoch 3/10
40/40 [==============================] - 80s 2s/step - loss: 1.0511 - val_loss: 0.0000e+00
Epoch 4/10
40/40 [==============================] - 86s 2s/step - loss: 1.0481 - val_loss: 0.0000e+00
Epoch 5/10
40/40 [==============================] - 88s 2s/step - loss: 1.0433 - val_loss: 0.0000e+00
Epoch 6/10
40/40 [==============================] - 85s 2s/step - loss: 1.0652 - val_loss: 0.0000e+00
Epoch 7/10
40/40 [==============================] - 86s 2s/step - loss: 1.1095 - val_loss: 0.0000e+00
Epoch 8/10
40/40 [==============================] - 82s 2s/step - loss: 1.0639 - val_loss: 0.0000e+00
Epoch 9/10
40/40 [==============================] - 73s 

In [20]:
import evaluate

metric = evaluate.load("squad")

In [21]:
from tqdm.auto import tqdm
import collections

import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [22]:
predictions= model.predict(tf_validation_set)
start_logits, end_logits = predictions['start_logits'], predictions['end_logits']
print(len(start_logits))
compute_metrics(start_logits, end_logits, tokenized_test_data, ds["test"])

14/14 [==============================] - 32s 398ms/step
54


  0%|          | 0/54 [00:00<?, ?it/s]

{'exact_match': 40.74074074074074, 'f1': 66.14986810339855}

In [ ]:
question = "Who developed the Abundant Life Planning Framework?"
context = "The Abundant Life Planning Framework, developed by business professor and serial entrepreneur Bob Nuebert from the University at Buffalo in New York, is designed to help students gain a clear vision of their life goals and the steps needed to achieve them."

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="abundant_life_planning_qa_model")
question_answerer(question=question, context=context)